# Python Flask API voor DungeonDaddies project
Code geschreven door Cédric Bonnemayers

In dit document deel en beschrijf ik de Python code achter de Flask API waar het DungeonDaddies project gebruik van maakt om verbinding te maken met de MySQL database.

<div class="alert alert-block alert-info" style="width:75%">
<b>Handig om te weten:</b> De code blokken in dit bestand zullen niet uitgevoerd kunnen worden omdat het afhankelijk is van de MySQL database en lokale verbinding. 
</div>

***
## Configuratie

Dit project is afhankelijk van een aantal packages en modules:

-  `flask` wordt gebruikt als API framework
-  `jsonify` wordt gebruikt om JSON code terug te geven aan de applicatie die de API oproept.
-  `request` wordt gebruikt om POST data te lezen.
-  `MySQL` en `MySQLdb.cursors` wordt gebruikt om verbinding te maken met de MySQL database en het lezen, schrijven en manipuleren van data.
-  `argon2` en `argon2.exceptions` wordt gebruikt om wachtwoorden op een veilige manier te hashen
-  `string` en `secrets` worden gebruikt om een random wachtwoord te genereren in de vorm van een string. 
-  `requests` zou gebruikt kunnen worden om een API call te doen naar een mailserver zoals mailtrap, voor het verzenden van een automatisch gegenereerd wachtwoord. Helaas is dat nog niet geïmplementeerd in de huidige versie.

In [4]:
from flask import Flask, jsonify, request
from flask_mysqldb import MySQL
import MySQLdb.cursors
from argon2 import PasswordHasher
from argon2.exceptions import VerifyMismatchError
import string
import secrets

# Not yet implemented
import requests

### Argon2 password hasher configuratie
De Argon2 wachtwoord hasher heeft een aantal configuratie instellingen nodig om te kunnen functioneren. Deze instellingen moeten goed afgestemd worden op de omgeving waar de code op draait (in dit geval een Netlab Ubuntu VM).
<br><br>
Als de instellingen te hoog zijn, kan het erg lang duren om een wachtwoord te hashen en te verifiëren. In het geval dat de inlogfunctie niet goed beveiligd is, kan dit ook bloot staan voor een DDOS aanval, omdat de CPU te hard moet werken.
<br><br>
Ik heb aan een AI gevraagd wat de beste instellingen zouden zijn voor een VM van 4GB RAM en 2 CPU cores. Dit is het resultaat:

In [ ]:
# Conservative settings for VM
ph = PasswordHasher(
    time_cost=2,        # 2 iterations
    memory_cost=65536,  # 64 MB memory usage
    parallelism=2,      # Use both CPU cores
    hash_len=32,        # 32-byte hash output
    salt_len=16         # 16-byte salt
)

### Configuratie Flask en MySQL server
In het onderste stuk code sla ik de MySQL configuratie op. Deze wordt gebruikt om verbinding te maken met de MySQL server die op dezelfde machine als de API draait, vandaar localhost.

In [ ]:
app = Flask(__name__)

# MySQL config
app.config['MYSQL_HOST'] = 'localhost'
app.config['MYSQL_USER'] = 'dungeonmaster'
app.config['MYSQL_PASSWORD'] = 'DungeonPass!@'
app.config['MYSQL_DB'] = 'dungeondaddies'
mysql = MySQL(app)

***
## Definiëring van functies
In dit kopje ga ik over de functies die later door de endpoints gebruikt kunnen worden.

### Functie om te checken of een account al bestaat
In het onderste stuk code heb ik een functie gemaakt waarmee ik kan checken of een account al bestaat met de gegeven paramteres.

**Parameters**
-  `account_id` wordt meegegeven als het account_id in de *users* tabel bekend is.
-  `email` wordt meegegeven als het emailadres in de *users* tabel bekend is.
-  `username` wordt meegegeven als de username in de *users* tabel bekend is.

Ik check voor iedere parameter of het meegegeven toen de functie werd opgeroept. Zo ja, voeg ik de meegegeven paramater toe aan de *conditions*, *params* en *found_by* arrays. Als er niks meegegeven wordt, geef ik een status terug die verteld dat er geen paramaters meegegeven zijn.

De query wordt dus pas aangemaakt en uitgevoerd als er minstens één paramater meegegeven is. In de query selecteer ik 1 rij van users waar de gegeven condities (paramaters) van toepassing zijn. Als er een rij gevonden kan worden, geef ik de status *EXISTS* mee en de paramater waarmee het gevonden is. Als er niks gevonden kan worden, geef ik de status *NOT_EXISTS* mee.

In [ ]:
def check_account_exists(account_id, email, username):
    cursor = mysql.connection.cursor()
   
    # Build dynamic query based on provided parameters
    conditions = []
    params = []
    found_by = []
   
    if account_id is not None:
        conditions.append("id = %s")
        params.append(account_id)
        found_by.append("id")
   
    if email is not None:
        conditions.append("email = %s")
        params.append(email)
        found_by.append("email")
   
    if username is not None:
        conditions.append("username = %s")
        params.append(username)
        found_by.append("username")
   
    # If no parameters provided, return NO_PARAMS error status
    if not conditions:
        cursor.close()
        return {"status": "NO_PARAMS"}
   
    # Join conditions with OR
    query = "SELECT 1 FROM users WHERE " + " OR ".join(conditions)
   
    try:
        cursor.execute(query, tuple(params))
        result = cursor.fetchone()
        
        if result is not None:
            return {"status": "EXISTS", "found_by": found_by}
        else:
            return {"status": "NOT_EXISTS"}
    finally:
        cursor.close()

### Functie voor het genereren van een random wachtwoord
In het onderste stuk code heb ik een functie gemaakt waarmee ik een random wachtwoord laat genereren. De functie genereerd een wachtwoord dat bestaat uit kleine letters, grote letters, nummers en speciale tekens zoals '-' en '_'. Een voorbeeld van een wachtwoord is: `2e7x5cm-rIH7PVj`

Het aantal kleine en grote letters en het aantal cijfers kan aangepast worden naar wens d.m.v. de variabelen. Ook kan de lengte van het wachtwoord aangepast worden. Dit getal kan natuurlijk niet lager zijn dan de drie criteria's bij elkaar opgeteld en daarom worden de waardes van de criteria's bij elkaar opgeteld en kan hier nog op toegevoegd worden. 

In het onderste voorbeeld is het maximale wachtwoord dus `4 + 4 + 4 + 3 = 15`

<div class="alert alert-block alert-info" style="width:fit-content">
<b>Tip:</b> Deze functie kan wél gestart en getest worden in het onderstaande codestuk.
</div>

In [16]:
def random_pw():
    min_amount_lowercase = 4
    min_amount_uppercase = 4
    min_amount_digits = 4
    max_pw_length = min_amount_lowercase + min_amount_uppercase + min_amount_digits + 3

    alphabet = string.ascii_letters + string.digits + '-_'
    while True:
        password = ''.join(secrets.choice(alphabet) for i in range(max_pw_length))
        if (sum(c.islower() for c in password) >= min_amount_lowercase
        and sum(c.isupper() for c in password) >= min_amount_uppercase
        and sum(c.isdigit() for c in password) >= min_amount_digits):
            return password


print(random_pw())

Z72eFKUivgZ6YX5


***
## Definiëring van API endpoints
In dit kopje ga ik over de verschillende endpoints van de API die door applicaties gebruikt kunnen worden.

### Registreren van een speler
In het onderste stuk code heb ik een endpoint gemaakt waarmee een gebruiker zich kan registreren in het systeem. Dit wordt gedaan d.m.v. een POST request en ik gebruik deze data om de gebruiker in de database toe te voegen.

In deze functie check ik:
-  Of alles ingevuld is
-  Of het ingevoerde emailadres of username al bestaat -> [check_account_exists()](#functie-om-te-checken-of-een-account-al-bestaat)

Als de ingevoerde data door alle checks gaat, wordt het wachtwoord gehashed met [ph](#argon2-password-hasher-configuratie).hash. Vervolgens voer ik een query uit die de ingevoerde data en het gehashte wachtwoord in de database opslaat. 

Tot slot geef ik een status code mee die aangeeft of het registreren gelukt is.

In [ ]:
@app.route('/register', methods=['POST'])
def create_user():
    try:
        # Assign data from POST request
        data = request.get_json()

        email = data.get('email')
        password = data.get('password')
        username = data.get('username')
        first_name = data.get('first_name')
        last_name = data.get('last_name')

        # Check if all fields are filled, return if not
        if not username or not email or not password or not first_name or not last_name:
            return jsonify({"error": "Missing required fields"}), 400

        # Check if email or username already exists, return if so
        account_exists = check_account_exists(account_id=None, email=email, username=username)

        if account_exists['status'] == "EXISTS":
            if account_exists['found_by'] is not None:
                return jsonify({"error": "Account already exists by" + account_exists['found_by']})
            
        # Hash password
        password_hash = ph.hash(password) 

        cursor = mysql.connection.cursor()

        # Query to insert user
        query = """
            INSERT INTO users (email, pw_hash, username, first_name, last_name) 
            VALUES (%s, %s, %s, %s, %s)
        """

        cursor.execute(query, (email, password_hash, username, first_name, last_name))

        mysql.connection.commit()
        cursor.close()

        return jsonify({'success': True, 'message': 'User created successfully'}), 201

    except Exception as e:
        print(f"Error creating user: {e}")
        return jsonify({'success': False, 'message': 'Server error'}), 500
    

### Inloggen van een gebruiker
In het onderste stuk code heb ik een endpoint gemaakt waarmee een gebruiker ingelogd kan worden. Dit wordt gedaan d.m.v. een POST request.

In deze functie check ik of alle velden ingevoerd zijn. Vervolgens voer ik een query uit waarin ik het ingevoerde emailadres gebruik om te kijken of het in de database bestaat. Zo ja, vergelijk en verifiëer ik het ingevoerde wachtwoord met het opgeslagen gehaste wachtwoord met [ph](#argon2-password-hasher-configuratie).verify. 

Tot slot geef ik een status code mee die aangeeft of het inloggen gelukt is.

In [ ]:
@app.route('/login', methods=['POST'])
def login():
    try:
        # Assign data from POST request
        data = request.get_json()
        email = data.get('email')
        password = data.get('password')
        
        # Check if all fields are filled, return if not
        if not email or not password:
            return jsonify({'success': False, 'message': 'Missing required fields'}), 400
        
        cursor = mysql.connection.cursor()

        # Query to fetch user credentials
        query = """
            SELECT id, email, pw_hash 
            FROM users 
            WHERE email = %s
        """

        cursor.execute(query, (email,))
        user = cursor.fetchone()
        cursor.close()
        
        # Check if user exists
        if user:
            account_id, user_email, stored_hash = user
            
            # Verify entered password with stored (hashed) password
            try:
                ph.verify(stored_hash, password)
                return jsonify({
                    'success': True, 
                    'message': 'Login successful',
                    'account_id': account_id,
                    'email': user_email
                })

            except VerifyMismatchError:
                return jsonify({'success': False, 'message': 'Invalid credentials'}), 401
        
        else:
            return jsonify({'success': False, 'message': 'Invalid credentials'}), 401
            
    except Exception as e:
        print(f"Login error: {e}")
        return jsonify({'success': False, 'message': 'Server error'}), 500

### Functie om het wachtwoord te resetten
In het onderste stuk code heb ik een endpoint gemaakt waarmee het wachtwoord van een gebruiker gereset kan worden. Ik doe dit via een POST request en maak gebruik van de [random_pw()](#functie-voor-het-genereren-van-een-random-wachtwoord) functie. 

Ik check of het emailadres geen lege waarde heeft en of het account bestaat via het emailadres -> [check_account_exists()](#functie-om-te-checken-of-een-account-al-bestaat).

Met het gegenereerd wachtwoord gebruik ik Argon2 om het wachtwoord te hashen. Vervolgens update ik de `password_hash` kolom van de gebruiker, in de `users` tabel, in de database. 

Nu zou de gebruiker in kunnen loggen met het nieuw gegenereerde wachtwoord. Maar de gebruiker weet dit wachtwoord natuurlijk nog niet. Er zou nu dus een mail naar de gebruiker uit moeten gaan die het nieuwe wachtwoord bevat. Het is mij echter nog niet gelukt om dit te implementeren.

In [ ]:
@app.route('/reset-pw', methods=['POST'])
def reset_pw():
    try:
        # Get data from POST
        data = request.get_json()
        email = data.get('email')

        # Check if all fields are filled, return if not
        if not email:
            return jsonify({'success': False, 'message': 'Missing required fields'}), 400
        
        # Check if account exists, return if not
        account_exists = check_account_exists(account_id=None, email=email, username=None)
        if account_exists['status'] == "NOT_EXISTS":
            return jsonify({'success': False, 'message': 'Account not found'}), 404

        random_password = random_pw() # Generate random password
        password_hash = ph.hash(random_password) # Hash password

        cursor = mysql.connection.cursor()

        # Query to update old password with new password using user email
        query = """
            UPDATE users
            SET pw_hash = %s
            WHERE email = %s;
        """

        cursor.execute(query, (password_hash, email))
        mysql.connection.commit()
        
        cursor.close()

        return jsonify({'success': True, 'message': 'Password reset successfully!'}), 201
        
    except Exception as e:
        print(f"Error updating password: {e}")
        return jsonify({'success': False, 'message': 'Server error'}), 500

### Gebruikerslijst ophalen
In het onderstaande stuk code heb ik een endpoint gemaakt de de gebruikerslijst ophaalt vanuit de database. Dit doe ik met een GET request.

In de query haal ik de volgende data op:
-  email
-  gebruikersnaam
-  voornaam
-  achternaam
-  tijdstip van registratie
-  tijdstip van aanpassing
-  rolnaam

Voorbeeld van query result:

<img src="img/fetch_users.png">

In [ ]:
@app.route('/fetch-users', methods=['GET'])
def fetch_users():
    try:
        cursor = mysql.connection.cursor(MySQLdb.cursors.DictCursor)

        # Query to fetch all users
        query = """
            SELECT u.email, u.username, u.first_name, u.last_name, u.created_at, u.updated_at, r.name as role_name
            FROM users u
            JOIN roles r ON u.role_id = r.id
        """
        cursor.execute(query)
            
        result = cursor.fetchall()
        cursor.close()
        
        return jsonify(result)

    except Exception as e:
        print(f"Error fetching users: {e}")
        return jsonify({'success': False, 'message': 'Server error'}), 500

### Bestaande gebruiker aanpassen
In het onderstaand stuk code heb ik een endpoint gemaakt waarmee gebruikersdata aangepast kan worden. Dit doe ik met een POST request.

Ik was nog bezig met een functie die checkt of het emailadres of gebruikersnaam al bestaat, maar dit bleek toch lastiger te zijn dan verwacht en dus ben ik er niet aan toegekomen om dit te realiseren.

Via deze endpoint kan de volgende data aangepast worden:
-  email
-  gebruikersnaam
-  voornaam
-  achternaam
-  rol



In [ ]:
@app.route('/edit-user', methods=['POST'])
def edit_user():
    try:
        # Assign data from POST request
        data = request.get_json()
        account_id = data.get('account_id')
        email = data.get('email')
        username = data.get('username')
        first_name = data.get('first_name')
        last_name = data.get('last_name')
        role_id = data.get('role_id')

        # Check if all fields are filled, return if not
        if not account_id or not email or not username or not first_name or not last_name or not role_id:
            return jsonify({"error": "Missing required fields"}), 400
        
        cursor = mysql.connection.cursor()

        # Query to update user by given data
        query = """
            UPDATE users SET
            email = %s, username = %s, first_name = %s, last_name = %s, role_id = %s, updated_at = NOW()
            WHERE id = %s
        """

        cursor.execute(query, (email, username, first_name, last_name, role_id, account_id))

        mysql.connection.commit()
        cursor.close()
        
        return jsonify({'success': True, 'message': 'User updated successfully'}), 201

    except Exception as e:
        print(f"Error updating user: {e}")
        return jsonify({'success': False, 'message': 'Server error'}), 500

### Verwijderen van een gebruiker
In het onderstaande stuk code heb ik een endpoint gemaakt waarmee een gebruiker verwijderd kan worden. Dit doe ik met een POST request.

Voor de zekerheid check ik of het account bestaat in de database om errors voor te komen. Vervolgens voer ik een query uit die de meegegeven user verwijderd uit de database.

Tot slot geef ik een status mee die aangeeft of de gebruiker is verwijderd.


In [ ]:
@app.route('/delete-user', methods=['POST'])
def delete_user():
    try:
        # Assing data from POST
        data = request.get_json()
        account_id = data.get('account_id')

        # Check if account exists, return if not
        account_exists = check_account_exists(account_id=account_id, email=None, username=None)

        if account_exists['status'] == "NO_PARAMS":
            return jsonify({"error": "No account id given"}), 400
        elif account_exists['status'] == "NOT_EXISTS":
            return jsonify({"error": "Account doesn't exist"}), 404

        cursor = mysql.connection.cursor()

        # Query to delete a user
        query = """
            DELETE FROM users
            WHERE id = %s
        """
        
        cursor.execute(query, (account_id,))

        mysql.connection.commit()
        cursor.close()
        
        return jsonify({'success': True, 'message': 'User deleted successfully'}), 201

    except Exception as e:
        print(f"Error deleting user: {e}")
        return jsonify({'success': False, 'message': 'Server error'}), 500

### Indienen van een nieuwe run
In het onderstaande stuk code heb ik een endpoint gemaakt waarmee een speler een nieuwe run kan indienen. Dit doe ik met een POST request.

Ik check of alle data een waarde heeft en of het meegegeven account bestaat. Vervolgens haal ik het laatste `player_run_number` van de speler op en tel hier 1 bij op, dit wordt het nieuwe `player_run_number`. Samen met de data uit de POST, kan ik een query maken die een nieuwe run toevoegd aan `runs` tabel. Tot slot voeg ik de maps uit de POST request toe aan de tabel `maps` en link deze met `run_id` aan de `runs` tabel.



In [ ]:
@app.route('/submit-run', methods=['POST'])
def submit_run():
    try:
        # Assign data from POST
        data = request.get_json()
        playtime = data.get('playtime')
        enemies_killed = data.get('enemies_killed')
        hits = data.get('hits')
        damage_dealt = data.get('damage_dealt')
        deaths = data.get('deaths')
        player_id = data.get('player_id')
        maps = data.get('maps', [])

        # Check if all fields are filled, return if not
        if not playtime or not enemies_killed or not hits or not damage_dealt or not deaths or not player_id or not maps:
            return jsonify({"error": "Missing required fields"}), 400

        # Check if account exists, return if not
        account_exists = check_account_exists(player_id, None, None)
        if account_exists['status'] == "NOT_EXISTS":
            return jsonify({"error": "Account not found"}), 404
        
        cursor = mysql.connection.cursor()

        # Query to get run number for player
        query_run_number = """
            SELECT COALESCE(MAX(player_run_number), 0) + 1 
            FROM runs 
            WHERE player_id = %s
        """

        cursor.execute(query_run_number, (player_id,))
        next_run_number = cursor.fetchone()[0]

        # Query to insert new run, using run fetched run number
        query_insert_run = """
            INSERT INTO runs (playtime, enemies_killed, hits, damage_dealt, deaths, player_id, player_run_number)
            VALUES (%s, %s, %s, %s, %s, %s, %s)
        """

        cursor.execute(query_insert_run, (playtime, enemies_killed, hits, damage_dealt, deaths, player_id, next_run_number))

        # Get last run id
        run_id = cursor.lastrowid

        # Insert map for each value in maps array
        for position, map_gen in enumerate(maps, start=1):
            query_insert_map = """
                INSERT INTO maps (map, run_id, run_position) 
                VALUES (%s, %s, %s)
            """
            cursor.execute(query_insert_map, (map_gen, run_id, position))

        mysql.connection.commit()
        cursor.close()

        return jsonify({'success': True, 'message': 'Run created successfully'}), 201

    except Exception as e:
        mysql.connection.rollback()
        print(f"Error creating run: {e}")
        return jsonify({'success': False, 'message': 'Server error'}), 500

### Ophalen van profieldata
In het onderstaande stuk code heb ik een endpoint gemaakt waarmee profieldata voor een speler opgehaald kan worden. Dit doe ik met een GET request. In de URL wordt het `player_id` meegegeven als integer.

Ik check of het account bestaat met het meegegeven `player_id`. Vervolgens voer ik een query uit die alle runs van de gegeven speler berekent:
-  Totaal aantal runs
-  Totaal aantal speeltijd
-  Totaal aantal vijanden verslagen
-  Totaal aantal keren geraakt
-  Totaal aantal schade geleverd
-  Totaal aantal keer af gegaan

Na deze berekening wordt het resultaat gegeven aan de applicatie.

Voorbeeld van een result:

<img src="img/fetch_profile.png" />

In [ ]:
@app.route('/fetch-profile/<int:player_id>', methods=['GET'])
def fetch_profile(player_id):
    try:

        # Check if account exists, return if not
        account_exists = check_account_exists(account_id=player_id, email=None, username=None)

        if account_exists['status'] == "NO_PARAMS":
            return jsonify({"error": "Missing required fields"}), 400
        elif account_exists['status'] == "NOT_EXISTS":
            return jsonify({"error": "Account not found"}), 404

        cursor = mysql.connection.cursor(MySQLdb.cursors.DictCursor)

        # Query to fetch all player stats by using:
        # COUNT() to calculate total amount of runs
        # SUM() to calculate stats over all runs
        query = """
            SELECT 
            COUNT(*) AS total_runs,
            SUM(playtime) AS total_playtime, 
            SUM(enemies_killed) AS total_enemies_killed, 
            SUM(hits) AS total_hits, 
            SUM(damage_dealt) AS total_damage_dealt, 
            SUM(deaths) AS total_deaths
            FROM runs
            WHERE player_id = %s
        """

        cursor.execute(query, (player_id,))
        result = cursor.fetchall()
        cursor.close()

        return jsonify(result)

    except Exception as e:
        print(f"Error fetching user: {e}")
        return jsonify({'success': False, 'message': 'Server error'}), 500

### Ophalen van runs van een speler
In het onderstaand stuk code heb ik een endpoint aangemaakt waarmee de runs van een speler opgehaald kunnen worden. Dit doe ik met een GET request.

Ik begin met checken of het account bestaat -> [check_account_exists()](#functie-om-te-checken-of-een-account-al-bestaat).

Vervolgens voer ik een query uit die de volgende data ophaalt:
-  datum van run
-  speeltijd van run
-  afgegaan of niet (deaths)
-  hoeveelste run
-  gebruikersnaam

Voorbeeld result van query:

<img src="img/fetch_runs.png" />

In [ ]:
@app.route('/fetch-runs/<int:account_id>', methods=['GET'])
def fetch_runs(account_id):
    try:
        
        # Check if account exists, return if not
        account_exists = check_account_exists(account_id=account_id, email=None, username=None)

        if account_exists['status'] == "NO_PARAMS":
            return jsonify({"error": "No account id given"}), 400
        elif account_exists['status'] == "NOT_EXISTS":
            return jsonify({"error": "Account doesn't exist"}), 404

        cursor = mysql.connection.cursor(MySQLdb.cursors.DictCursor)
        
        # Query to select all runs and username with player_id
        query = """
            SELECT r.date, r.playtime, r.deaths, r.player_run_number, u.username
            FROM runs r
            JOIN users u ON r.player_id = u.id
            WHERE r.player_id = %s;
        """

        cursor.execute(query, (account_id,))
        result = cursor.fetchall()

        cursor.close()

        return jsonify(result)
    
    except Exception as e:
        mysql.connection.rollback()
        print(f"Error fetching maps: {e}")
        return jsonify({'success': False, 'message': 'Server error'}), 500